# Run Ilastik segmentation and TrackMate tracking

This notebook runs Ilastik segmentation and TrackMate tracking

To work correctly, it requires the following input:
- A metadata ***.csv*** containing information on every sample to be processed
- A 5D (ctzyx) ***.tiff*** file of the raw imaging data




In [5]:
from behav3d.preprocessing.segmentation.ilastik_segmentation import run_ilastik_segmentation
from behav3d.preprocessing.tracking.trackmate_tracking import (
    run_trackmate_tcells, 
    run_trackmate_organoids,
    run_trackmate
)
import pandas as pd
from pathlib import Path

### Load metadata and supply Ilastik paths


In [3]:
### Path for the output files generated by the pipeline
output_dir = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/BEHAV3D_run/behav3d_output"
# PATH TO THE METADATA CSV
metadata_csv_path = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/BEHAV3D_run/metadata.csv"

# PATH TO ILASTIK
ilastik_path = "/Applications/ilastik-1.4.0b27-OSX.app/Contents/ilastik-release/run_ilastik.sh"

# MODEL PATHS
ilastik_pixel_classifier_model = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_pixel_class.ilp"
ilastik_organoid_segmentation_model = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_organoid_class.ilp"
ilastik_tcell_segmentation_model = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_tcell_class.ilp"
ilastik_organoid_postprocessing_model = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_object_splitter.ilp"
ilastik_tcell_postprocessing_model = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_object_splitter.ilp"

### Read in the metadata file and display it
metadata = pd.read_csv(metadata_csv_path)
print("Metadata provided:")
metadata

Metadata provided:


,sample_name,organoid_line,tcell_line,exp_nr,well,dead_dye_channel,dead_dye_threshold,contact_threshold,pixel_distance_xy,pixel_distance_z,distance_unit,time_interval,time_unit,raw_image_path,tcell_segments_path,organoid_segments_path,tcell_track_csv
0,AIM_MB2_Exp58_Img003_donor899,10T,CART_WT1,1,well0,3,3000,3.6,1.38,1.38,μm,2,m,/Users/samdeblank/Documents/1.projects/BHVD_BE...,/Users/samdeblank/Documents/1.projects/BHVD_BE...,/Users/samdeblank/Documents/1.projects/BHVD_BE...,/Users/samdeblank/Documents/1.projects/BHVD_BE...


### Check correct usage of metadata.csv

Checks the supplied metadata .csv for:
- Are all columns filled in?
- Does teh raw_image path exist?

In [6]:
assert not any(metadata.isna().any()), "Some column values have not been supplied. Make sure you correctly supply values for all columns in the metadata .csv"

for rowidx, sample_metadata in metadata.iterrows():
    sample_name = sample_metadata['sample_name']
    assert Path(sample_metadata["raw_image_path"]).exists(), f"The image_path supplied for Row {rowidx+1} '{sample_name}' does not exist"
print("Metadata file is OK!")

Metadata file is OK!


---

# Performing ilastik segmentation

---

Uses the pretrained supplied ilastik models for pixel classification, object detection and object postprocessing (object splitting) for both T cells and Organoids/Tumor cells

*Duration: 2 hours*<br>
*On filesize: 30Gb*

Pixel classifier: ~1 hour


In [9]:
run_ilastik_segmentation(
    output_dir=output_dir,
    metadata=metadata,
    ilastik_path = ilastik_path,
    ilastik_pix_clas_model = ilastik_pixel_classifier_model,
    ilastik_org_seg_model = ilastik_organoid_segmentation_model,
    ilastik_tcell_seg_model = ilastik_tcell_segmentation_model,
    ilastik_org_postproc_model = ilastik_organoid_postprocessing_model,
    ilastik_tcell_postproc_model = ilastik_tcell_postprocessing_model, 
    verbose=False
    )

--------------- Using existing raw_img .h5: AIM_MB2_Exp58_Img003_donor899 ---------------
/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/BEHAV3D_run/behav3d_output/images/AIM_MB2_Exp58_Img003_donor899/AIM_MB2_Exp58_Img003_donor899_ilastik_input.h5
--------------- Segmenting: AIM_MB2_Exp58_Img003_donor899 ---------------
- Running the ilastik pixel classifier...
Running command:
/Applications/ilastik-1.4.0b27-OSX.app/Contents/ilastik-release/run_ilastik.sh --headless --project='/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_pixel_class.ilp' --output_format='hdf5' --output_internal_path /probabilities --output_filename_format=/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/BEHAV3D_run/behav3d_output/images/AIM_MB2_Exp58_Img003_donor899/AIM_MB2_Exp58_Img003_donor899_probabilities.h5 --output_axis_order=ctzyx --export_source='Probabilities' /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/B

---

# Performing TrackMate tracking

---

Applies TrackMate for the tracking of the T cells and organoids/tumor cells over all timepoints

In [ ]:
run_trackmate(
    output_dir=output_dir, 
    metadata=metadata, 
    verbose=False, 
    cell_type="tcells"
    )
run_trackmate(
    output_dir=output_dir, 
    metadata=metadata, 
    verbose=False, 
    cell_type="organoids"
    )